In [0]:
##Importing some necessary files.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

In [0]:
##Loading train and test files from google.drive

!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



     |████████████████████████████████| 993kB 3.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
##Training data..
train_file = drive.CreateFile({'id':'15T0LRYTQphP0Emu3_Ri9TBSBd8zZTgVo'})
train_file.GetContentFile('train.csv')

In [0]:
!ls

adc.json  sample_data  test.csv  train.csv


In [0]:
##Loading training data in to running env...
train = pd.read_csv("train.csv")
print("Shape of the training file is {}".format(train.shape))

train.head()


Shape of the training file is (330575, 7)


,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,27270,27270,7,01/12/10 8:26,2.55,PX,85123AY
1,27270,27270,7,01/12/10 8:26,3.39,PX,71053R
2,27270,27270,9,01/12/10 8:26,2.75,PX,84406BH
3,27270,27270,7,01/12/10 8:26,3.39,PX,84029GV
4,27270,27270,7,01/12/10 8:26,3.39,PX,84029EX


# EDA

In [0]:
data = pd.melt(train.set_index('CustomerID')['StockCode'].apply(pd.Series).reset_index(), 
             id_vars=['CustomerID'],
             value_name='StockCode') \
    .groupby(['CustomerID', 'StockCode']) \
    .agg({'StockCode': 'count'}) \
    .rename(columns={'StockCode': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'StockCode': 'productId'})

In [0]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [0]:
data_dummy['purchase_dummy'].value_counts()

1    271031
Name: purchase_dummy, dtype: int64

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='CustomerID', columns='productId')

In [0]:
df_matrix.head()

productId,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,...,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [0]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(256976, 3)


,CustomerID,productId,scaled_purchase_freq
38,25470,10002F,0.0
99,50949,10002F,0.0
114,56340,10002F,0.0
132,64539,10002F,0.0
140,68229,10002F,0.0


In [0]:
data_norm['scaled_purchase_freq'].describe()

count    256976.000000
mean          0.081394
std           0.209182
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: scaled_purchase_freq, dtype: float64

In [0]:
!pip install turicreate

In [0]:

import turicreate as tc
from sklearn.model_selection import train_test_split
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [0]:
train_data.head()

CustomerID,productId,purchase_count
205488,20754G,1
176418,22670P,1
443556,90175DM,1
286317,82551H,1
59040,22326L,1
194508,22496R,1
514485,22629C,1
386406,22727B,2
7290,22766P,1
402696,22732E,1


In [0]:
# constant variables to define field names include:
user_id = 'CustomerID'
item_id = 'productId'
users_to_recommend = list(set(train['CustomerID']))
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output datase

In [0]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
        
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name,user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 972 users and 3769 items.

Data prepared in: 0.226455s

216824 observations to process; with 3769 unique items.

+------------+-----------+--------------------+------+
| CustomerID | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |  85099BJ  | 2.2557172557172556 |  1   |
|     0      |  81953BF  |        2.0         |  2   |
|     0      |   22197J  | 1.8520408163265305 |  3   |
|     0      |   84879M  | 1.8121827411167513 |  4   |
|     0      |   23442T  |        1.8         |  5   |
|     0      |   22386V  |      1.78125       |  6   |
|     0      |  90199CA  |        1.75        |  7   |
|     0      |   22960K  | 1.7359550561797752 |  8   |
|     0      |   23298Q  | 1.7262247838616716 |  9   |
|     0      |   22457C  | 1.7245989304812834 |  10  |
|   32769    |   22423U  | 2.3326133909287257 |  1   |
|   32769    |  85099BJ  | 2.2557172557172556 |  2   |
|   32769    |   20725N  | 2.0349127182044886 |  3   |
|   32769    |   47566Y  | 2.009367681498829  |  4   |
|   32769    |  81953BF  |        2.0         |  5   |
|   32769 

In [0]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 971 users and 3757 items.

Data prepared in: 0.215681s

216824 observations to process; with 3757 unique items.

+------------+-----------+-------+------+
| CustomerID | productId | score | rank |
+------------+-----------+-------+------+
|     0      |   23345S  |  1.0  |  1   |
|     0      |   23166P  |  1.0  |  2   |
|     0      |   85111R  |  1.0  |  3   |
|     0      |   22144E  |  1.0  |  4   |
|     0      |   22427Q  |  1.0  |  5   |
|     0      |   22632O  |  1.0  |  6   |
|     0      |   23263V  |  1.0  |  7   |
|     0      |   23280V  |  1.0  |  8   |
|     0      |   23445C  |  1.0  |  9   |
|     0      |   22619G  |  1.0  |  10  |
|   32769    |   23166P  |  1.0  |  1   |
|   32769    |   85111R  |  1.0  |  2   |
|   32769    |   22144E  |  1.0  |  3   |
|   32769    |   22427Q  |  1.0  |  4   |
|   32769    |   22632O  |  1.0  |  5   |
|   32769    |   23263V  |  1.0  |  6   |
|   32769    |   23280V  |  1.0  |  7   |
|   32769    |   23445C  |  1.0  |  8   |
|   32769    |   22135T  |  1.0  |  9   |
|   32769    |   22619G  |  1.0  |  10  |
|   24579    |   23166P  |  1.0  |

In [0]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 970 users and 2494 items.

Data prepared in: 0.221277s

205580 observations to process; with 2494 unique items.

+------------+-----------+--------------------+------+
| CustomerID | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |   21690A  |        0.5         |  1   |
|     0      |   84548X  |        0.5         |  2   |
|     0      |   21816Y  |        0.4         |  3   |
|     0      |  81950BS  |        0.4         |  4   |
|     0      |  90163AR  | 0.3333333333333333 |  5   |
|     0      |  90065AP  | 0.3333333333333333 |  6   |
|     0      |  72369AK  | 0.3333333333333333 |  7   |
|     0      |   21392H  | 0.3333333333333333 |  8   |
|     0      |   21692C  | 0.3333333333333333 |  9   |
|     0      |  84800SH  | 0.3333333333333333 |  10  |
|   32769    |   21690A  |        0.5         |  1   |
|   32769    |   84548X  |        0.5         |  2   |
|   32769    |   21816Y  |        0.4         |  3   |
|   32769    |  81950BS  |        0.4         |  4   |
|   32769    |  90163AR  | 0.3333333333333333 |  5   |
|   32769 

In [0]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 972 users and 3769 items.

Data prepared in: 0.238602s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.83726s

+------------+-----------+---------------------+------+
| CustomerID | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |  85099BJ  | 0.35181105850286004 |  1   |
|     0      |   22697A  |  0.3442507255031335 |  2   |
|     0      |   22698C  |  0.3043483741955407 |  3   |
|     0      |   22197J  |  0.2891385129519871 |  4   |
|     0      |   22960K  |  0.278026434675607  |  5   |
|     0      |   22457C  | 0.27010036205232835 |  6   |
|     0      |   22384W  | 0.24566093581983942 |  7   |
|     0      |   22411U  | 0.24276164231613337 |  8   |
|     0      |   20724F  |  0.2297011652968565 |  9   |
|     0      |   22699D  |  0.2274866594310893 |  10  |
|   32769    |  85099BJ  |  0.300057199719834  |  1   |
|   32769    |   22697A  |  0.2889028738622796 |  2   |
|   32769    |   22698C  | 0.27968724623118363 |  3   |
|   32769    |   22423U  |  0.2598721948388505 |  4   |
|   32769    |   22993E  |  0.2548749014122845 |

In [0]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 971 users and 3757 items.

Data prepared in: 0.250066s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.847996s

+------------+-----------+---------------------+------+
| CustomerID | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |  85099BJ  |  0.2817227173207411 |  1   |
|     0      |   21212D  |  0.2666340608205368 |  2   |
|     0      |   22666I  |  0.2551191880631803 |  3   |
|     0      |  85123AY  |  0.2549898875738258 |  4   |
|     0      |   22961C  | 0.24113619038418158 |  5   |
|     0      |   21931F  | 0.23755754085619057 |  6   |
|     0      |   20724F  | 0.22411387691746898 |  7   |
|     0      |   22411U  | 0.22037610789733147 |  8   |
|     0      |   22197J  | 0.21948742332743176 |  9   |
|     0      |   22697A  | 0.21802683530458764 |  10  |
|   32769    |  85099BJ  |  0.3121627668539683 |  1   |
|   32769    |   22423U  | 0.30961647858986485 |  2   |
|   32769    |   21212D  |  0.2958217049256349 |  3   |
|   32769    |   22961C  | 0.28088530057515854 |  4   |
|   32769    |   23209X  |  0.2776714379970844 |

In [0]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 205580 observations with 970 users and 2494 items.

Data prepared in: 0.232515s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.45378s

+------------+-----------+-----------------------+------+
| CustomerID | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|     0      |  85099BJ  |  0.008018909036651138 |  1   |
|     0      |   22197J  | 0.0071691755176514615 |  2   |
|     0      |   21212D  | 0.0070126444794410886 |  3   |
|     0      |   84879M  |  0.005604107481564662 |  4   |
|     0      |   22411U  | 0.0055282647295515665 |  5   |
|     0      |   22355U  |  0.005500907583754192 |  6   |
|     0      |   22457C  |  0.005329735519350037 |  7   |
|     0      |   20724F  |  0.005264455264852952 |  8   |
|     0      |   23298Q  |  0.00513896900554036  |  9   |
|     0      |   22666I  |  0.004871608906014021 |  10  |
|   32769    |   20616Z  |  0.007198045128270199 |  1   |
|   32769    |  84251GR  |  0.006233690600646169 |  2   |
|   32769    |   23539T  |  0.006233690600646169 |  3   |
|   32769    |   22583A  |  0.006233690600646169 |  4   |
|   32769    |

In [0]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216824 observations with 972 users and 3769 items.

Data prepared in: 0.221357s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.35023s

+------------+-----------+--------------------+------+
| CustomerID | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |  85099BJ  | 2.252705102314537  |  1   |
|     0      |  81953BF  |        2.0         |  2   |
|     0      |   22197J  | 1.8469323615835833 |  3   |
|     0      |   84879M  | 1.8119087774573235 |  4   |
|     0      |   23442T  | 1.8005689640763183 |  5   |
|     0      |   22386V  | 1.7794132025545624 |  6   |
|     0      |  90199CA  | 1.7496431115511302 |  7   |
|     0      |   22960K  | 1.7341399700138653 |  8   |
|     0      |   23298Q  | 1.7248661666350815 |  9   |
|     0      |   22457C  | 1.7227597444775153 |  10  |
|   32769    |   22423U  | 2.319864990286276  |  1   |
|   32769    |  85099BJ  | 2.239458643646258  |  2   |
|   32769    |   20725N  | 2.027221008241879  |  3   |
|   32769    |   47566Y  | 2.0021570953298324 |  4   |
|   32769    |  81953BF  |        2.0         |  5   |
|   32769 

In [0]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216824 observations with 971 users and 3757 items.

Data prepared in: 0.216084s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.14314s

+------------+-----------+-------+------+
| CustomerID | productId | score | rank |
+------------+-----------+-------+------+
|     0      |   23345S  |  0.0  |  1   |
|     0      |   23166P  |  0.0  |  2   |
|     0      |   85111R  |  0.0  |  3   |
|     0      |   22144E  |  0.0  |  4   |
|     0      |   22427Q  |  0.0  |  5   |
|     0      |   22632O  |  0.0  |  6   |
|     0      |   23263V  |  0.0  |  7   |
|     0      |   23280V  |  0.0  |  8   |
|     0      |   23445C  |  0.0  |  9   |
|     0      |   22619G  |  0.0  |  10  |
|   32769    |   23166P  |  0.0  |  1   |
|   32769    |   85111R  |  0.0  |  2   |
|   32769    |   22144E  |  0.0  |  3   |
|   32769    |   22427Q  |  0.0  |  4   |
|   32769    |   22632O  |  0.0  |  5   |
|   32769    |   23263V  |  0.0  |  6   |
|   32769    |   23280V  |  0.0  |  7   |
|   32769    |   23445C  |  0.0  |  8   |
|   32769    |   22135T  |  0.0  |  9   |
|   32769    |   22619G  |  0.0  |  10  |
|   24579    |   23166P  |  0.0  |

In [0]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 205580 observations with 970 users and 2494 items.

Data prepared in: 0.212656s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.07823s

+------------+-----------+---------------------+------+
| CustomerID | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |   21690A  | 0.49998377483020456 |  1   |
|     0      |   84548X  |  0.4999360874179722 |  2   |
|     0      |   21816Y  |         0.4         |  3   |
|     0      |  81950BS  |  0.3999707544496817 |  4   |
|     0      |   85189T  | 0.33433769654858025 |  5   |
|     0      |   21393V  | 0.33333333333333337 |  6   |
|     0      |  72369AK  | 0.33333333333333337 |  7   |
|     0      |   21392H  | 0.33333333333333337 |  8   |
|     0      |   21692C  | 0.33333333333333337 |  9   |
|     0      |  84751BL  | 0.33333333333333337 |  10  |
|   32769    |   21690A  |         0.5         |  1   |
|   32769    |   84548X  |         0.5         |  2   |
|   32769    |   21816Y  |  0.3999880963250211 |  3   |
|   32769    |  81950BS  |  0.3999007190528669 |  4   |
|   32769    |  72369AK  | 0.33333333333333337 |

In [0]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [0]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.30095036958817367 | 0.006661611580714211 |
|   2    | 0.24392819429778273 | 0.011922541299614451 |
|   3    |  0.2189369940161919 | 0.015325915060604907 |
|   4    | 0.19799366420274533 | 0.017808764226943417 |
|   5    |  0.1894403379091872 | 0.020255192776740656 |
|   6    | 0.18268215417106637 | 0.02247730058376422  |
|   7    | 0.17951425554382264 | 0.025147102638212182 |
|   8    |  0.1814941921858502 | 0.028705245004105314 |
|   9    | 0.18420743869529524 | 0.03182723774380091  |
|   10   |  0.1859556494192186 | 0.034824056629645946 |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.5240659632429618

Per User RMSE (best)
+------------+-----------

In [0]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=3)
recom.print_rows(10)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 271031 observations with 972 users and 3810 items.

Data prepared in: 0.246361s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.15516s

+------------+-----------+---------------------+------+
| CustomerID | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |  85099BJ  |  0.372740737678718  |  1   |
|     0      |   22197J  |  0.3444147321485704 |  2   |
|     0      |   22699D  | 0.34062284563293904 |  3   |
|   32769    |  85099BJ  |  0.4265949355096233 |  1   |
|   32769    |   22197J  |  0.3906760732738339 |  2   |
|   32769    |   20725N  |  0.3777158211688606 |  3   |
|   24579    |   22086O  |  0.216906381977929  |  1   |
|   24579    |  85123AY  | 0.19505339860916138 |  2   |
|   24579    |   21212D  | 0.19446545177035862 |  3   |
|   343557   |   22699D  |  0.3756618609925582 |  1   |
+------------+-----------+---------------------+------+
[2916 rows x 4 columns]



In [0]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['CustomerID', 'recommendedProducts']].drop_duplicates() \
        .sort_values('CustomerID').set_index('CustomerID')
    if print_csv:
        df_output.to_csv('recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [0]:
df_output = create_output(pear_norm, users_to_recommend, 3, print_csv=True)
print(df_output.shape)
df_output.head()

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(972, 1)


,recommendedProducts
CustomerID,
0,21690A|84548X|21816Y
900,84548X|21690A|81950BS
1800,84548X|21690A|21816Y
2790,84548X|21690A|81950BS
4590,84548X|21690A|21816Y


In [0]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [0]:
##generating output...
customer_recomendation(32769)

recommendedProducts    21690A|84548X|21816Y
Name: 32769, dtype: object